In [2]:
import torch
import timm

from torchvision import models, transforms
from PIL import Image
import urllib
import glob
import numpy as np


c:\Users\docto\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the pretrained models

inception_V3_model = models.inception_v3(pretrained=True)
inception_V3_model.eval()  # Set the model to inference mode

inception_V4_model = timm.create_model('inception_v4', pretrained=True)
inception_V4_model.eval()

resnet50_model = models.resnet50(pretrained = True)
resnet50_model.eval()

resnet101_model = models.resnet101(pretrained=True)
resnet101_model.eval()

vgg16_model = models.vgg16(pretrained=True)
vgg16_model.eval()

vgg_19_model = models.vgg19(pretrained = True)
vgg_19_model.eval()


# Define the image preprocessing
preprocess = transforms.Compose([
    transforms.Resize(299),  # Resize the image to 299x299
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load and preprocess an image from Tiny ImageNet
image_path = 'data\\tiny-imagenet-200\\val\\images\\val_'  
#image_path = 'data\\tiny-imagenet-200\\val\\images\\*'  
image_list = []
tag_list  = []
for imageCount in range(0,10000): # this way I can set it to a lower number to get faster results
    currpath = image_path+str(imageCount)+'.JPEG'
#for iamgeName in glob.glob(image_path):
    image = Image.open(currpath).convert('RGB')
    image = preprocess(image)
    image = image.unsqueeze(0)  # Add a batch dimension
    image_list.append(image)
with open('data\\tiny-imagenet-200\\val\\val_annotations.txt', 'r') as f:
    for line in f:
        y = line.split()
        tag_list.append(y[1])
f.close()
with open('data\\tiny-imagenet-200\\words.txt', 'r') as f:
    wnid_list = []
    names_list = []
    for line in f:
        y = line.split('\n')[0].split('\t')
        if y!=['']:
            wnid_list.append( y[0])
            names = y[1].split(', ')
    
            names_list.append(names)
        #print(y)
f.close()

c:\Users\docto\anaconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\docto\anaconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\docto\anaconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet5

In [4]:
currpath = image_path+str(0)+'.JPEG'
#for iamgeName in glob.glob(image_path):
image = Image.open(currpath).convert('RGB')
image = preprocess(image)
image = image.unsqueeze(0)  # Add a batch dimension

In [6]:
url, filename = ("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", "imagenet_classes.txt")
urllib.request.urlretrieve(url, filename) 
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

def get_Accuracy(model):
    correct_count=0
    img_count = 0
    for currImage in image_list:
        
        # Perform inference
        with torch.no_grad():
            outputs = model(currImage)
        
        # The output has unnormalized scores. To get probabilities, apply softmax.
        probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
        
        # let's do top 1
        top_prob, top_catid = torch.topk(probabilities,1)
        catid = top_catid[0].item()
        prob = top_prob[0].item()
        #print(categories[catid])
        for currlist in names_list:
            if categories[catid] in currlist:
            
                predWNID = wnid_list[names_list.index(currlist)]

                if predWNID in tag_list:

                    correct_count+=1
        img_count+=1
    return round(correct_count/img_count,4)
    # code in case we want to do top 5
    # Display the top 5 categories per the original ImageNet labels
    # Note: You would need to map these to Tiny ImageNet classes manually as discussed
    # top5_prob, top5_catid = torch.topk(probabilities, 5)
    # for i in range(top5_prob.size(0)):
    #     print(f"{i+1}: {top5_catid[i].item()} with probability {top5_prob[i].item()}")
    #     print(categories[top5_catid[i]], top5_prob[i].item())

vgg16_acc = get_Accuracy(vgg16_model)
print("VGG16 accuracy:",vgg16_acc)
vgg19_acc = get_Accuracy(vgg_19_model)
print("VGG19 accuracy:",vgg19_acc)
resnet50_acc = get_Accuracy(resnet50_model)
print("Resnet50 accuracy",resnet50_acc)
resnet101_acc = get_Accuracy(resnet101_model)
print("Resnet100 accuracy",resnet101_acc)
v3_acc = get_Accuracy(inception_V3_model)
print("Inception V3 Accuracy",v3_acc)
v4_acc = get_Accuracy(inception_V4_model)
print("Inception V4 Accuracy",v4_acc)

VGG16 accuracy: 0.245
VGG19 accuracy: 0.253
Resnet50 accuracy 0.333
Resnet100 accuracy 0.4204
Inception V3 Accuracy 0.498
Inception V4 Accuracy 0.4684


@misc{szegedy2016inceptionv4,
      title={Inception-v4, Inception-ResNet and the Impact of Residual Connections on Learning}, 
      author={Christian Szegedy and Sergey Ioffe and Vincent Vanhoucke and Alex Alemi},
      year={2016},
      eprint={1602.07261},
      archivePrefix={arXiv},
      primaryClass={cs.CV}
}